# GREGoR QC notebook: check keys/indices across data tables
created by: Marsha Wheeler <br>
last edited: 08-12-23

In [ ]:
# install modules
# %pip install terra-pandas

In [ ]:
# import modules
import os
import io
import pandas as pd
import terra_pandas as tp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec
import seaborn as sns
from functools import reduce

In [ ]:
project = os.environ['WORKSPACE_NAMESPACE']
workspace = os.environ['WORKSPACE_NAME']
bucket = os.environ['WORKSPACE_BUCKET'] + "/" 
current_upload_cycle = 'U03'

print("Terra Billing project: " + project)
print("Workspace: " + workspace)
print("Workspace storage bucket: " + bucket)
print("Current GREGoR Upload Cycle: " + current_upload_cycle)

In [ ]:
# functions
def readDatatable(data_table, project, workspace): 
    new_table = tp.table_to_dataframe(data_table, workspace_namespace=project, workspace=workspace)
    return new_table

## Get indices and foreign keys in data tables

In [ ]:
tables = ['participant', 'family', 'phenotype', 'analyte', 'experiment_dna_short_read', 
          'experiment_rna_short_read', 'aligned_dna_short_read', 'aligned_rna_short_read']

In [ ]:
primary_keys = []
for i in tables: 
    data_table = readDatatable(i, project, workspace)
    keys = (i + '.' + data_table.index.name)
    primary_keys.append(keys)

In [ ]:
foreign_keys = []
for i in tables: 
    data_table = readDatatable(i, project, workspace)
    for c in data_table.columns:
        #print(c)
        if c.endswith('id'):
            keys = (i + '.' + c)
            foreign_keys.append(keys)

In [ ]:
new_dict = {  "id_tables_01" : ["family_id", "participant", "family" ],
            "id_tables_02" : ["participant_id", "participant", "phenotype"],
            "id_tables_03" : ["participant_id", "participant", "analyte"],
            "id_tables_04" : ["analyte_id", "analyte", "experiment_dna_short_read"], 
            "id_tables_05" : ["analyte_id", "analyte", "experiment_rna_short_read"], 
            "id_tables_06" : ["experiment_dna_short_read_id", "experiment_dna_short_read", "aligned_dna_short_read"], 
            "id_tables_07" : ["experiment_rna_short_read_id", "experiment_rna_short_read", "aligned_rna_short_read"]
           }

In [ ]:
for key, item in new_dict.items():
    print('Checking the following index and U03 data tables: ', (str(item)).strip('[]'))
    table1 = readDatatable(item[1], workspace = workspace, project = project)
    table2 = readDatatable(item[2], workspace = workspace, project = project)
    print('Number of ' + item[1] + ': ' +  str(table1.shape[0]))
    print('Number of ' + item[2] + ': ' +  str(table2.shape[0]))
    df = table1.merge(table2, on = item[0], how = 'outer', indicator = True)
    res = pd.DataFrame(df['_merge'].value_counts())
    res.index = res.index.str.replace('left_only', item[1]).str.replace('right_only', item[2])

    print(res)
    print('-------------------------------------')

## Follow-up on foreign key mismatches

In [ ]:
# read in AnVIL tables from combined consortium U03 workspace
participant = readDatatable('participant', project, workspace)
family =readDatatable('family', project, workspace)
phenotype =readDatatable('phenotype', project, workspace)
analyte =readDatatable('analyte', project, workspace)
experiment_dna_short_read =readDatatable('experiment_dna_short_read', project, workspace)
experiment_rna_short_read =readDatatable('experiment_rna_short_read', project, workspace)
aligned_dna_short_read =readDatatable('aligned_dna_short_read', project, workspace)
aligned_rna_short_read =readDatatable('aligned_rna_short_read', project, workspace)

### __1 'family_id' not present in 'participant' table__

In [ ]:
# which family id is missing in the participant table
family[~family.index.isin(participant['family_id'])]

### __1254 participant ids not present in the phenotype table__

In [ ]:
# which participant ids are missing in the phenotype table
missing_pheno = participant[~participant.index.isin(phenotype['participant_id'])]
missing_pheno['affected_status'].value_counts()

In [ ]:
missing_pheno['proband_relationship'].value_counts()

### 75 participant ids not in the analyte table

In [ ]:
missing_analyte = participant[~participant.index.isin(analyte['participant_id'])]
missing_analyte['affected_status'].value_counts()

In [ ]:
missing_analyte['proband_relationship'].value_counts()

### 196 analyte ids not in the experiment_DNA_short_read table

In [ ]:
missing_DNA = analyte[~analyte.index.isin(experiment_dna_short_read['analyte_id'])]
missing_DNA['analyte_type'].value_counts()

In [ ]:
missing_DNA_2 = missing_DNA[missing_DNA['analyte_type'] == 'DNA']
missing_DNA_2.index

### 2454 analyte ids not in the experiment_RNA_short_read table

In [ ]:
missing_RNA = analyte[~analyte.index.isin(experiment_rna_short_read['analyte_id'])]
missing_RNA['analyte_type'].value_counts()

In [ ]:
missing_RNA_2 = missing_RNA[missing_RNA['analyte_type'] == 'RNA']
missing_RNA_2.index

### 6 experiment DNA short read ids not present in aligned DNA short read table

In [ ]:
missing_aligned_DNA = experiment_dna_short_read[~experiment_dna_short_read.index.isin(aligned_dna_short_read['experiment_dna_short_read_id'])]
missing_aligned_DNA['analyte_id'].index

_**FYI: PMGRC-576-576-0_LS4429508_SQ6070 is no longer present in the data set_